In [1]:
import pandas as pd
import numpy as np

from pyomo.environ import *

from data_cleaning import hub_occ_dict
from data_cleaning import heatdays_df
from data_cleaning import cengeo_pop_dict
from data_cleaning import bg_ces_df

dist_to_hub_df = pd.read_csv('data/distmatrix_contracosta.csv')
dist_to_hub_df.set_index('Unnamed: 0', inplace = True)
dist_to_hub_df.index.name = None

### Define Model

_Create model and variable indices_

In [2]:
model = ConcreteModel()

model.idx_cengeos = Set(initialize = dist_to_hub_df.index)
model.idx_hubs = Set(initialize = dist_to_hub_df.columns)

_Create set of blockgroup, hub pairs within five miles of each other_

In [3]:
def filter_to_nearby_hubs(model, cg, hub):
    return not np.isnan(dist_to_hub_df.loc[cg, hub])

model.idx_cg_hub_pairs = Set(initialize = model.idx_cengeos*model.idx_hubs, filter = filter_to_nearby_hubs)

In [4]:
included_cengeos = []
included_hubs = []

for pair in model.idx_cg_hub_pairs:
    if pair[0] not in included_cengeos:
        included_cengeos.append(pair[0])
    if pair[1] not in included_hubs:
        included_hubs.append(pair[1])
        
model.idx_cengeos = Set(initialize = included_cengeos)
model.idx_hubs = Set(initialize = included_hubs)

    'pyomo.core.base.set.OrderedScalarSet'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>).
    This is usually indicative of a modelling error. To avoid this warning,
    use block.del_component() and block.add_component().
    'pyomo.core.base.set.OrderedScalarSet'>) on block unknown with a new
    Component (type=<class 'pyomo.core.base.set.AbstractOrderedScalarSet'>).
    This is usually indicative of a modelling error. To avoid this warning,
    use block.del_component() and block.add_component().


### Define Parameters

_Pairwise distance indexed on (blockgroup, hub) tuple_

In [5]:
def get_cg_hub_distance(model, cg, hub):
    return dist_to_hub_df.loc[cg, hub]

model.param_cg_hub_dist = Param(model.idx_cg_hub_pairs, initialize = get_cg_hub_distance)

_Dictionary of blockgroups within range for each hub_

In [6]:
cgs_at_hub = {hub: [] for hub in model.idx_hubs}

for hub in cgs_at_hub:
    for pair in model.idx_cg_hub_pairs:
        if pair[1] == hub:
            cgs_at_hub[hub].append(pair[0])

hubs_with_no_cgs = [key for key in cgs_at_hub.keys() if len(cgs_at_hub[key]) == 0]

for hub in hubs_with_no_cgs:
    del cgs_at_hub[hub]
    
model.param_cgs_at_hub = Param(model.idx_hubs, within=Any, initialize=cgs_at_hub)

### Define Variables

_Is this site a hub?_

In [7]:
model.var_hub_yn = Var(model.idx_hubs, initialize = 0, within = Binary)

_What proportion of a blockgroup's population is served by this hub?_

In [8]:
model.var_prop_cg_at_hub = Var(model.idx_cg_hub_pairs, initialize = 0.0, bounds = (0.0, 1.0))

_How many people are served by this hub?_

In [9]:
def ppl_at_hub(model):
    return(sum(model.var_prop_cg_at_hub[cg, hub]*cengeo_pop_dict[cg] for cg in model.param_cgs_at_hub[hub]))

model.var_hub_pop = Var(model.idx_hubs, initialize = ppl_at_hub, domain = NonNegativeReals)

def ppl_served_at_hub(model, hub):
    tot_ppl_served = 0

    for cg in model.idx_cengeos:
        if (cg, hub) in model.idx_cg_hub_pairs:
            ppl_served = model.var_prop_cg_at_hub[cg, hub]*cengeo_pop_dict[cg]
            tot_ppl_served = ppl_served + tot_ppl_served
        else:
            tot_ppl_served = tot_ppl_served
    
    return(tot_ppl_served)

model.var_hub_pop = Var(model.idx_hubs, initialize = ppl_served_at_hub, domain = NonNegativeReals)

_How many people in this block group are assigned to a hub?_

# del
def cg_total_assigned(model, cg):
    tot_ppl_served = 0

    for hub in model.idx_hubs:
        if (cg, hub) in model.idx_cg_hub_pairs:
            ppl_served = model.var_prop_cg_at_hub[cg, hub]*cengeo_pop_dict[cg]
            tot_ppl_served = ppl_served + tot_ppl_served
        else:
            tot_ppl_served = tot_ppl_served
    
    return(tot_ppl_served)

model.var_cg_pop_assigned = Var(model.idx_cengeos, initialize = cg_total_assigned, domain = NonNegativeReals)

### Define Objective

_Minimize aggregate travel distance_

In [10]:
min_agg_dist = sum(model.param_cg_hub_dist[cg, hub] * cengeo_pop_dict[cg] * model.var_prop_cg_at_hub[cg, hub] 
                   for cg, hub in model.idx_cg_hub_pairs)

model.obj_min_agg_dist = Objective(expr = min_agg_dist, sense = minimize)

### Define Constraints

_Hub cannot serve more people than it has capacity for_

In [11]:
def serve_less_than_hub_occ(model, hub):
    return(model.var_hub_pop[hub] <= hub_occ_dict[hub])

model.con_max_occ = Constraint(model.idx_hubs, rule = serve_less_than_hub_occ)

_Construct a set number of hubs_

In [12]:
model.con_max_hubs = Constraint(expr = sum(model.var_hub_yn[hub] for hub in model.idx_hubs) == 3)

_No one is served at a location if it is not deemed a hub_

In [13]:
def open_only(model, cg, hub):
    return(model.var_prop_cg_at_hub[cg, hub] <= model.var_hub_yn[hub])

model.con_open_only = Constraint(model.idx_cg_hub_pairs, rule = open_only)

_Open hubs must be at least 90% capacity_

def fill_open_hubs(model, hub):
    return(hub_occ_dict[hub])

model.param_hub_occ = Param(model.idx_hubs, initialize = get_hub_occ)

_Meet a certain portion of demand in area of interest_

In [14]:
sum_ppl_served = sum(model.var_hub_pop[hub] for hub in model.idx_hubs) 
sum_area_pop = sum(cengeo_pop_dict[cg] for cg in model.idx_cengeos)

model.con_min_coverage = Constraint(expr = sum_ppl_served >= 0.5*sum_area_pop)

_Prioritize CalEnviroScreen populations_

model.con_serve_ces = ConstraintList()

for cg in model.idx_cengeos:
    cg_ces_score = float(bg_ces_df.loc[bg_ces_df['GISJOIN'] == cg, 'SCORE_PCTL_CI_BG'])

    if cg_ces_score >= 75.0:
        model.con_serve_ces.add(expr = model.var_cg_pop_assigned[cg] >= 0.95*cengeo_pop_dict[cg])

_Do not meet more than 100% of demand_

In [15]:
model.con_demand_max = ConstraintList() #Constrain the TOTAL proportion of people assigned a hub to be 100 percent

for cg in model.idx_cengeos:
    # Only add constraint for cengeos that have hubs, may return error if "if" statement ommited
    if sum([pair[0]==cg for pair in model.idx_cg_hub_pairs]) >= 1:
        model.con_demand_max.add(expr = sum(model.var_prop_cg_at_hub[pair] for pair in model.idx_cg_hub_pairs if pair[0]==cg)<=1)

### Solve Model

In [16]:
import pyomo.opt as pyopt

In [17]:
pyopt.SolverFactory('glpk').solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': 0.0, 'Upper bound': 0.0, 'Number of objectives': 1, 'Number of constraints': 24762, 'Number of variables': 24688, 'Number of nonzeros': 72388, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': '1', 'Number of created subproblems': '1'}}, 'Error rc': 0, 'Time': 0.4530820846557617}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [18]:
res_hub_pop = [model.var_hub_pop[hub].value for hub in model.idx_hubs]
res_hub_yn = [model.var_hub_yn[hub].value for hub in model.idx_hubs]

res_prop_cg_at_hub_list = []

for cg in model.idx_cengeos:
    cg_dict = dict()
    for pair in model.idx_cg_hub_pairs:
        if pair[0]==cg:
            cg_dict[pair[1]] = model.var_prop_cg_at_hub[pair].value
    res_prop_cg_at_hub_list.append(cg_dict)
    
res_prop_cg_at_hub = pd.DataFrame(res_prop_cg_at_hub_list)

In [22]:
[hub for hub in model.var_hub_yn if model.var_hub_yn[hub].value > 0]

['106479449', '268810770', '131837314']

In [27]:
print([prop for prop in res_prop_cg_at_hub['106479449'] if prop > 0])

print([prop for prop in res_prop_cg_at_hub['268810770'] if prop > 0])

print([prop for prop in res_prop_cg_at_hub['131837314'] if prop > 0])

[]
[]
[]


In [30]:
res_prop_cg_at_hub[(res_prop_cg_at_hub > 0).any(axis = 1)]

,204345954,292031477,31553746,31555561,31556371,31556463,31556529,31556616,31556696,31557528,...,825084384,825084471,28006024,28113846,42991416,42989617,42989621,823961879,824734791,9387379


model_solver = pyopt.SolverFactory('glpk')

model_results = model_solver.solve(model, tee=True)

results_df = pd.DataFrame(index = pd.MultiIndex.from_tuples(model.idx_cg_hub_pairs, names = ['cengeos', 'hubs']))